# BeachBoys BikeShare

Deze workshop is een ML gefocuste versie van de BeachBoys BikeSharing case. In deze case onderzoeken we één van hun bedrijfsproblemen en zorgen we voor een slimme oplossing aan de hand van data.<br>
BeachBoys BikeShare is een fiets sharing bedrijf in de Bay Area. Ze hebben verschillende stations met fietsenstallingen waar klanten op elk moment een fiets kunnen huren. Ze kunnen vervolgens de fiets terugbrengen naar eender welk van hun stations.<br>
Het probleem waar we op gaan inzoomen vandaag is het optimaal gebruik van hun fietsen. Elk station heeft een bepaalde hoeveelheid stallingen, wanneer er geen fietsen beschikbaar zijn, verliezen we klanten aangezien ze geen trip kunnen maken. Wanneer het station vol is verliezen we klantentevredenheid, aangezien de klanten niet langer kunnen afstappen waar ze willen.<br>
Momenteel is er een wagen die voltijds rondrijdt en extra fietsen ophaalt/levert waar nodig, deze rijdt echter louter op goed gevoel. Het zou handig zijn moesten we deze kunnen sturen naar de locaties waar de hulp nodig is. De auto heeft natuurlijk tijd nodig om te navigeren tussen de verschillende stations, we zouden dus een periode op voorhand al moeten kunnen voorspellen waar deze drukke stations zich bevinden. Een ideale taak voor machine learning!<br><br>


Korte vraag: Wat voor machine learning model hebben we hiervoor nodig? <br> <br>

## Inhoud
- Importeer Libraries
- Data Analyse
   - Station Data
   - Trip Data
   - Weer Data
- Data Transformatie
- Model Trainen
   - Baseline wiskundig model
   - Lineaire regressie
   - Ridge regressie
   - Decision Tree (Beslissingboom)
   - Random Forest
   - Gradient Boosting
   - Multilayer Perceptron
- Visualiseren


Tijden het analyseren van de data gaan we jullie zelf laten experimenteren met de verschillende beschikbare data en het plotten ervan om inzichten te creëren.<br>
Het transformeren van de data is verdere oefening op het gebruik van pandas om data voor te bereiden voor machine learning modellen.<br>
Vervolgens gaan we enkele modellen trainen en evalueren. Hier kunnen we experimenteren met de invloed van verschillende parameters, alsook met het effect van de features.


# Voor we beginnen

We beginnen met eerst de nodige data en libraries in te laden.

In [ ]:
%pip install -r ../requirements.txt

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt 
import os
import numpy as np
import shap


In [ ]:
# Dit commando opent een popup waar je je moet inloggen op je azure account
#!az login

Haal data uit een storage container

In [ ]:
# Nu we zijn ingelogt kunnen we verbinden met de storage account en vervolgens de data uit de correcte container halen
"""
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

try:
    default_credential = DefaultAzureCredential() # regelt de authenticatie

    connection_string = "https://mowworkshopprep.blob.core.windows.net"
    blob_service_client = BlobServiceClient(connection_string,default_credential) #verbinden met de storage account
    container_name = "workshopdata"
    container_client = blob_service_client.get_container_client(container_name) #verbinden met de container

    local_path = "../data"

    blob_list = container_client.list_blobs() #lees alle bestanden in de container
    for blob in blob_list: # Ga over alle bestanden (blobs) en schrijf ze weg
        blob_client = container_client.get_blob_client(blob.name)    
        local_file_path = f"{local_path}/{blob.name}"
        with open(local_file_path, "wb") as local_file:
            local_file.write(blob_client.download_blob().readall())
    
    # Controleer of alle bestanden gedownload zijn
    required_files = ["station_data.csv", "trip_data.csv", "weather_data.csv", "previousValues.csv"]
    for file_name in required_files:
        file_path = os.path.join(local_path, file_name)
        if not os.path.isfile(file_path):
            raise FileNotFoundError(f"{file_name} not found in the local folder.")
    print("All required files found in the local folder.")
except Exception as e:
    print(f"An error occurred: {str(e)}")
"""

Haal data uit vanuit een data asset op machine learning studio

In [ ]:
datastoreURI = "azureml://subscriptions/749321d1-c59b-435d-b86b-d37cb334a686/resourcegroups/rg-mow-prepworkshop/workspaces/MOW-WorkshopPrer/datastores/workspaceblobstore/paths/UI/2024-01-26_162051_UTC/"
required_files = ["station_data.csv", "trip_data.csv", "weather_data.csv", "previousValues.csv"]
for file in required_files:
  df = pd.read_csv(f"{datastoreURI}{file}")
  df.to_csv(f"../data/{file}", index=False)


In [ ]:
os.listdir("../data")

# Analyseer de Tabellen

De eerste stap is kijken welke data beschikbaar is, en of hier enkele opvallende trends in te vinden zijn. <br>
Het eerste bestand is 'station_data.csv', elke rij hier vertelt meer over een bepaald station: de locatie, het aantal plaatsen voor fietsen,... <br>
Het tweede bestand bevat een rij voor elke geregistreerde trip, en geeft aan waar en wanneer deze gestart/ beëindigt zijn. Dit is ook de grootste tabel aangezien het informatie bevat voor elke gemaakte trip.<br>
De laatste tabel is informatie over het weer in en rond San Francisco. 

## Station data

In [ ]:
station_data = pd.read_csv("../data/station_data.csv")
station_data.head()

Elk station bevat verschillende gegevens:
- id: een unieke getal voor elk station
- Name: Naam van het station
- Lat: breedtegraad
- Long: lengtegraad
- Dock count: aantal plaatsen voor fietsen in het station
- City: De stad waar het station zich bevind

In [ ]:
column_types = station_data.dtypes
print(column_types)

Het type 'object' betekent string in dit geval. Alle kolommen staan reeds in de juiste vorm.

### Aantal Stations per stad

In [ ]:
station_data['City'].value_counts().plot(kind="bar")
plt.xlabel("Stad")
plt.ylabel("Aantal Stations")

### Totaal aantal fiestplaatsen per stad

In [ ]:
DocksPerCity = station_data.groupby(['City'])['Dock Count'].sum().reset_index()
plt.bar(DocksPerCity['City'],DocksPerCity['Dock Count'])
plt.xlabel("Stad")
plt.ylabel("Totaal aantal fietsplaatsen")

***Nu is het aan Jullie, denk aan statestieken die we nog kunnen gebruiken. Gebruik de datasets naar keuze***

## Trip data

In [ ]:
trip_data = pd.read_csv("../data/trip_data.csv")
trip_data.head()

Elke trip die een persoon maakt bevat de volgende gegevens:
- Trip ID: een uniek nummer voor elke trip
- Start Date: start tijd en datum van de trip
- Start Station: Waar is de trip begonnen
- End Date: eind tijd en datum van de trip
- End Station: Waar is de trip beëindigd
- Subscriber Type: Wat soort abonnement heeft de gebruiker

In [ ]:
column_types = trip_data.dtypes
print(column_types)

Hier zien we dat de start en eind tijd in string formaat staan. Hierdoor kunnen we niet redeneren over tijd, wat een belangrijk aspect is voor het voorspellen van het aantal fietsen. We moeten deze dus eerst transformeren.

In [ ]:
trip_data['Start Date'] = pd.to_datetime(trip_data['Start Date'], format='%d/%m/%Y %H:%M')
trip_data['End Date'] = pd.to_datetime(trip_data['End Date'], format='%d/%m/%Y %H:%M')
print(trip_data.dtypes)

In [ ]:
trip_data["tripDuration"] = (trip_data['End Date']-trip_data['Start Date']).dt.total_seconds()/60
trip_data

In [ ]:
minDate = trip_data['Start Date'].min()
minDate

In [ ]:
maxDate = trip_data['End Date'].max()
maxDate

We zien nu dat we ongeveer 1 jaar aan data hebben van September 2014 tot en met Augustus 2015.

Nu zou het ook interessant zijn om naar trends in de tijd te kunnen kijken. Zijn er spitsuren? Is het drukker tijdens het weekend of in de week? Is er een winterstop? Al deze vragen zijn relevant voor het voorspellen waar veel fietsen nodig zijn. <br>
Momenteel staan de tijdstippen opgeslagen in 1 kolom, onbewust kunnen wij dit in onze gedachten al opsplitsen in jaren, maanden, dagen, uren,...<br>
Maar het machine learning model kan dit niet, ook maakt dit het plotten meer ingewikkeld. Daarom splitsen we nu eerst de tijdstippen op.

In [ ]:
# Splits Start Date op
trip_data['StartYear'] = trip_data['Start Date'].dt.year
trip_data['StartMonth'] = trip_data['Start Date'].dt.month
trip_data['StartDay'] = trip_data['Start Date'].dt.day
trip_data['StartWeekday'] = trip_data['Start Date'].dt.weekday  # Maandag is 0 en zondag is 6
trip_data['StartHour'] = trip_data['Start Date'].dt.hour
trip_data['StartMinute'] = trip_data['Start Date'].dt.minute

# Splits End Date op
trip_data['EndYear'] = trip_data['End Date'].dt.year
trip_data['EndMonth'] = trip_data['End Date'].dt.month
trip_data['EndDay'] = trip_data['End Date'].dt.day
trip_data['EndWeekday'] = trip_data['End Date'].dt.weekday  # Maandag is 0 en zondag is 6
trip_data['EndHour'] = trip_data['End Date'].dt.hour
trip_data['EndMinute'] = trip_data['End Date'].dt.minute
trip_data.head()

***Nu is het aan Jullie, denk aan statestieken die we nog kunnen gebruiken. Gebruik de datasets naar keuze***

## Weer data

In [ ]:
weather_data = pd.read_csv("../data/weather_data.csv")
weather_data.head()

In [ ]:
column_types = weather_data.dtypes
print(column_types)

In [ ]:
weather_data['Date'] = pd.to_datetime(weather_data['Date'], format='%d/%m/%Y')
weather_data = weather_data[weather_data['Zip'] == 94107]
weather_data['Events'] = weather_data['Events'].fillna('None')


***Nu is het aan Jullie, denk aan statestieken die we nog kunnen gebruiken. Gebruik de datasets naar keuze***

# Data transformatie

***Dit deel bevat enkele lijnen code die nog ingevuld moeten worden, let opde TODO's en vul in waar ... staat.***

Nu dat we een beter begrip hebben van de gegevens, is het tijd om de data klaar te maken voor een machine learning model. <br>
De data moet 2 onderdelen bevatten voor we ons model kunnen trainen. <br> 
1. Eerst moeten we een 'Target' hebben, de waarde die we willen voorspellen. In deze opdracht willen we ervoor zorgen dat op elk moment fietsen beschikbaar zijn voor de klanten. We weten niets over het absolute aantal fietsen op een station, dus deze waarde is moeilijk te voorspellen. Wat we wel weten is hoeveel fietsen aankomen en vertrekken op een bepaald station. Hierdoor kunnen we een netto 'verandering aan fietsen' berekenen en ook voorspellen. Deze verandering per uur bekijken lijkt een goeie target, per minuut is te ambitieus en geeft ook geen tijd voor de volgwagen om nieuwe fietsen te leveren waar nodig, en aangezien de meeste trips korter zijn dan één uur gaat per dag een te groot venster zijn. Plaatsen waar we grote veranderingen voorspellen kunnen we de volgauto naartoe sturen, om extra fietsen af/op te laden.<br><br>
2. Ten tweede hebben we features nodig die het model kan gebruiken om onze target te voorspellen. De exploratie die we juist gedaan hebben kan hiervoor nuttig zijn alsook onze eigen intuitie. We zagen bijvoorbeeld dat er 2 piek uren rond 8 en rond 17, het uur is dus een belangrijke feature om de 'verandering aan fietsen' te voorspellen. Andere gegevens zoals de dag van de week, de maand, welk station, het weer zijn allemaal zaken die invloed kunnen hebben op 'verandering aan fietsen'.

In [ ]:
trip_data.head()

Dit is de data waar we mee zijn geëindigd na de voorgaande analyse. We kunnen deze eerst wat opschonen en de 
- start/end date 
- tripDuration 
- duration_bins
- Start/End minuten 
verwijderen (We voorspellen per uur dus de minuten zijn niet meer nodig).

In [ ]:
columnsToDrop = ['Start Date','End Date','tripDuration','StartMinute','EndMinute']
trip_data = #TODO verwijder de bovenstaande kolommen, hint: gebruik de drop functie

trip_data

Als alles goed zit, hebben we nu 14 kolommen aan data.

Om de netto 'verandering in fietsen' op een bepaald station te voorspellen hebben we 2 onderdelen nodig, de fietsen die aankomen in dit station (End Station) en degene die vertrekken vanaf dit station (Start Station)

In [ ]:
IncomingBikes = trip_data.groupby(['Start Station', 'StartYear','StartMonth','StartDay','StartHour']).size().reset_index(name='incoming')
IncomingBikes.head()

We weten niet of er op elk uur een fiets vertrekt of aankomt op een bepaald station. Dus we maken eerst een dataset aan die voor elk station alle uren bevat.

In [ ]:
date_range = pd.date_range(start=minDate, end=maxDate, freq='H')
date_df = pd.DataFrame(date_range, columns=['datetime'])
date_df['dateIndex'] = range(len(date_df)) # We voegen een tijdsindex toe, deze gaat ons later helpen

stations_df = pd.DataFrame(trip_data['Start Station'].unique(), columns=['Station']) # Extraheer alle verschillende stations
stationTime_df = pd.merge(date_df,stations_df,how='cross')
stationTime_df

In [ ]:
#Splits de datum terug op
#TODO: splits stationTime_df per jaar
stationTime_df['year'] =  ...
#TODO: splits stationTime_df per maand
stationTime_df['month'] = ...
#TODO: splits stationTime_df per dag
stationTime_df['day'] = ...
#TODO: splits stationTime_df per weekdag
stationTime_df['weekday'] = ...
#TODO: splits stationTime_df per uur
stationTime_df['hour'] = ...
stationTime_df= stationTime_df.drop('datetime',axis=1)
stationTime_df.head()

In [ ]:
IncomingBikes = pd.merge(stationTime_df, IncomingBikes, left_on=['Station', 'year','month','day','hour'],right_on=['Start Station','StartYear','StartMonth','StartDay','StartHour'] , how='left')
IncomingBikes = IncomingBikes.drop(['Start Station','StartYear','StartMonth','StartDay','StartHour'],axis=1)

#TODO: Vul de waarde in waar we lege rijen mee opvullen. Als er geen rij beschikbaar was, hoeveel fietsen kwamen er dan aan
IncomingBikes['incoming'] = IncomingBikes['incoming'].fillna(...).astype(int)
IncomingBikes.head()

Nu we de aankomende fietsen hebben, doen we hetzelfde voor de vertrekkende fietsen.

In [ ]:
#TODO: Op welke kolommen gaan we groeperen?
OutGoingBikes = trip_data.groupby([...]).size().reset_index(name='outgoing')

#TODO: welke tabellen moeten we nu verbinden?
OutGoingBikes = pd.merge(..., ..., left_on=['Station', 'year','month','day','hour'],right_on=['End Station','EndYear','EndMonth','EndDay','EndHour'] , how='left')
OutGoingBikes = OutGoingBikes.drop(['End Station','EndYear','EndMonth','EndDay','EndHour'],axis=1)

#TODO: Vul de waarde in waar we lege rijen mee opvullen. Als er geen rij beschikbaar was, hoeveel fietsen zijn er dan vertrokken?
OutGoingBikes['outgoing'] = OutGoingBikes['outgoing'].fillna(...).astype(int)
OutGoingBikes.head()

Nu moeten we nog het verschil nemen tussen beide om aan het netto verandering van fietsen te komen.

In [ ]:
ChangeInBikes = pd.merge(IncomingBikes,OutGoingBikes,on=['dateIndex','Station','year','month','day','hour','weekday'],how= 'left')
ChangeInBikes.head()

In [ ]:
#TODO: bereken de netto verandering aan fietsen
ChangeInBikes['netBikeChange'] = ...
ChangeInBikes.head()

NetBikeChange is onze target value en dus de waarde die we willen voorspellen, de overige kolommen zijn al enkele features die we kunnen gebruiken om de voorspelling te maken. Natuurlijk de incoming en outgoing waardes zijn onbekend tijdens het voorspellen, dus die verwijderen we nog eerst.

In [ ]:
ChangeInBikes= ChangeInBikes.drop(['incoming','outgoing'],axis=1)
ChangeInBikes.head()

Nu kunnen we even nadenken welke andere features nog van pas zouden komen.
- Het aantal fiestenstalling in een station
- de Stad waarin het station zich bevindt
- Is het weekend?
- Weer events (regen,mist,...)
- Temperatuur
- Verandering tijdens de laatste uren

### Is het weekend?

In [ ]:
#TODO: gebruik de isin functie om weekend dagen aan te duiden in een feature (cast daarna ook de feature naar een int via '.astype(int)')
ChangeInBikes['IsWeekend'] = ChangeInBikes[...]. ...
ChangeInBikes.head()

### In welke stad is het station? En hoeveel fiets plaatsen zijn er?

In [ ]:
ChangeInBikes = pd.merge(ChangeInBikes, station_data, left_on='Station', right_on='Id',how='left')

#TODO: verwijder de onnodige kolommen van de data via de drop functie (Id, Lat, Long en Name)
ChangeInBikes = ...
ChangeInBikes.head()

We kunnen de stad niet zomaar gebruiken, niet alle modellen kunnen werken met categorie waarden. We zullen deze eerst veranderen via one-hot encoding.

In [ ]:
#TODO: verander de namen van de steden in 1 woord door spaties te verandering in een laag streepje.
ChangeInBikes['City'] = ChangeInBikes['City'].str.replace(...,...)

citiesEncoded = pd.get_dummies(ChangeInBikes['City'], prefix='City')
ChangeInBikes = pd.concat([ChangeInBikes, citiesEncoded], axis=1)

ChangeInBikes= ChangeInBikes.drop('City',axis=1)
ChangeInBikes.head()

### Temperatuur en Weer events?

Voor we de 2 features over het weer kunnen toevoegen moeten we de datums van de weer tabel aanpassen. Beide datums moeten in dezelfde vorm staan zodat we de tabellen kunnen verbinden.

In [ ]:
weather_data['year'] = weather_data['Date'].dt.year
weather_data['month'] = weather_data['Date'].dt.month
weather_data['day'] = weather_data['Date'].dt.day

#TODO: Transformeer de gemiddelde temperatuur van farenheit naar celcius
weather_data['Mean TemperatureC'] = (... - 32) * 5/9
#TODO: Rond de nieuwe temperaturen af op 2 cijfers na de komma
weather_data['Mean TemperatureC'] = weather_data['Mean TemperatureC']. ...

weather_data.head()

In [ ]:
weather_data_features = weather_data[['year','month','day','Mean TemperatureC','Events']]
weather_data_features.head()

De events moeten ook opgesplitst worden per waarde zoals de steden.

In [ ]:
#TODO: One hot encode de weer events (tip: gelijkaardig als bij de steden, en prefix met 'Event')
weatherEncoded = ...
weather_data_features = pd.concat([weather_data_features, weatherEncoded], axis=1)
weather_data_features= weather_data_features.drop('Events',axis=1)
weather_data_features.head()

In [ ]:
#TODO: verbind de ChangeInBikes en weather_data_features tabellen via de pd.merge functie.
ChangeInBikes = ...
ChangeInBikes.head()

### Wat gebeurde er tijdens de vorige uren?

Een volgend feature dat kan helpen, is een zeer populaire bij het voorspellen van waarden over tijd (Time series predictions). Namelijk de waarde van het vorige uur (of van de vorige uren). Als we terugdenken aan onze piek uren, herinneren we ons dat drukke periodes niet plots opdaagden, maar geleidelijk op en afbouwt. 

Hier komt de tijdsindex terug van pas

In [ ]:
def getNetChangeFromPreviousHour(df,dateIndex,station,n=1):
    if (dateIndex-n) > 0:
        prevNetChange = df.loc[(df['dateIndex'] == dateIndex-n) & (df['Station'] == station), 'netBikeChange'].values[0] #loc zoekte specifieke rijen in de data
    else:
        prevNetChange = 0
    return prevNetChange

Dit zijn zwaardere functies aangezien we over elke rij moeten gaan en vervolgens voor elke rij de vorige waardes opzoeken in de tabel. Daarom is er ook een bestandje toegevoegd dat de nodige waardes al bevat. Door deze te linken aan onze tabel kunnen we wat tijd besparen.

In [ ]:
#ChangeInBikes['NetBikeChange-1'] = ChangeInBikes.apply(lambda row: getNetChangeFromPreviousHour(ChangeInBikes,row['dateIndex'], row['Station']), axis=1)
#ChangeInBikes['NetBikeChange-2'] = ChangeInBikes.apply(lambda row: getNetChangeFromPreviousHour(ChangeInBikes,row['dateIndex'], row['Station'],2), axis=1)

De volgende functie laad het bestand in en voegt de waardes toe aan de feature tabel.

In [ ]:
previousValues = pd.read_csv("../data/previousValues.csv")
ChangeInBikes = pd.merge(ChangeInBikes, previousValues, on=['dateIndex','Station'],how='left')
ChangeInBikes.head()

Hier voegen we de veranderingen tijdens de vorige 2 uren toe, we kunnen nog meer van deze historische waardes toevoegen, maar we moeten telkens het nut hiervan afwegen. Elk feature dat we toevoegen maakt ons model iets zwaarder. Hoeveel fietsen er 5 uur geleden zijn bijgekomen gaat weinig tot geen invloed hebben op hoeveel fietsen we nu gaan zien aankomen.<br><br>
Voor we dit deel afmaken verwijderen we nog even de 'dateIndex', deze was handig voor de vorige waardes toe te voegen maar heeft nu geen nut meer.

In [ ]:
ChangeInBikes = ChangeInBikes.drop('dateIndex',axis=1)
ChangeInBikes.head()

We eindigen met 22 kolommen

# Model trainen

Het trainen van de modellen bevat geen zelf in te vullen lijnen, maar wel verschillende parameters waarmee geexperimenteerd kan worden. Alsook het aantal features waar we aan kunnen sleutelen. Onderstaande code laat ons toe om een aantal features te kiezen om mee verder te gaan. Indien er genoeg tijd is, raad ik zeker aan om terug naar dit punt te komen en hier mee te experimenteren.

In [ ]:
ChangeInBikes.columns

In [ ]:
# Experimenteer gerust  met de features in de volgende lijsten
weatherColomns= ["Mean TemperatureC","Event_Fog","Event_Fog-Rain","Event_None","Event_Rain","Event_Rain-Thunderstorm"]
timeColumns = ["year","month","day","weekday","hour","IsWeekend","NetBikeChange-1","NetBikeChange-2"]
stationColumns = ["Station","Dock Count","City_Mountain_View","City_Palo_Alto","City_Redwood_City","City_San_Francisco","City_San_Jose"]


selectedColumns = weatherColomns+timeColumns+stationColumns
selectedColumns.append("netBikeChange")
ML_dataset= ChangeInBikes[selectedColumns]
ML_dataset.head()

Voor we kunnen beginnen modelleren en voorspellingen maken, moeten we de data nog één keer aanpassen. We moeten hem eerst opsplitsen in 2 groepen:<br>
De Trainingsdata, die we gebruiken om onze modellen te leren hoe ze onze targetwaarde kunnen voorspellen. Dit is meestal ook het grootste deel van de data, zodat het model betere predicties kan maken.<br>
De test data, deze wordt apart gehouden tot het einde om te controleren hoe goed ons model werkt op ongeziene data.<br>
Vaak is er nog een derde categorie, de validatie data, deze helpt om bepaalde parameters voor de modellen te kiezen. Vandaag gaan we geen extra parameters afwegen dus we kunnen deze data gebruiken om een grondigere eindevaluatie te voeren.<br><br>
Hoeveel data we gebruiken voor elke groep, moeten we zelf afwegen. Een vuistregel is 80/10/10, 80% van de data voor training, 10% voor validatie en 10% voor het testen.<br>
Vaak delen we de dataset quasi willekeurig op in de verschillende groepen, op deze manier voorkomen we dat er bijvoorbeeld nooit regen gezien werd in de trainingsdata. Maar in tijdserie voorspellingen, zoals de data waar we momenteel mee bezig zijn, wordt er vaak gesplitst op een bepaalde datum aangezien dit beter de realiteit reflecteert.


In dit geval hebben we bijna een jaar aan data, laten we de eerste 9 maanden gebruiken om te trainen en de overige 2,5 om te testen. Dit zou betekenen dat we splitsen vanaf 1 juni. Alles ervoor is onze training set, alles erna is onze test set

In [ ]:
training_df = ML_dataset[(ML_dataset['year']==2014) | (ML_dataset['month']< 6)]
training_df

In [ ]:
test_df =  ML_dataset[(ML_dataset['year']==2015) & (ML_dataset['month']>= 6)]
test_df

### Eerste wiskundige versie

Een goede tip tijdens het trainen van machine learning modellen is om te beginnen met een zeer simpele versie. Deze hoeft niet eens gebruik te maken van machine learning en kan gewoon wiskundig zijn. Op deze manier hebben we een basis, waartegen we de meer geavanceerde versies kunnen vergelijken.<br>
In dit geval zou zo een basis bijvoorbeeld kunnen zijn om de gemiddelde 'verandering aan fietsen' te berekenen en altijd deze te voorspellen.<br>
Wij gaan voor een iets geavanceerdere versie gaan. Aangezien we de 'verandering aan fietsen' gedurende de vorige 2 momenten hebben, kunnen we de vorige veranderingen berekenen en deze verder zetten tijdens het volgende uur.



In [ ]:
def derivativePrediction(lastHour,twoHoursAgo):
    return lastHour + (lastHour-twoHoursAgo) 

In [ ]:
test_deriv_df = test_df.copy()
test_deriv_df['predicted'] = test_df.apply(lambda row: derivativePrediction(row['NetBikeChange-1'], row['NetBikeChange-2']), axis=1)
test_deriv_df.head()

In [ ]:
from sklearn.metrics import mean_squared_error, root_mean_squared_error

test_MSE = mean_squared_error(test_deriv_df['netBikeChange'],test_deriv_df['predicted'])
print(f"baseline: de MSE error op de test set is: {test_MSE}")
test_RMSE = root_mean_squared_error(test_deriv_df['netBikeChange'],test_deriv_df['predicted'])
print(f"baseline: de MSE error op de test set is: {test_RMSE}")

Deze mean square error geeft aan hoeveel fietsen we gemiddeld naast de eigenlijke verandering zitten. Deze baseline geeft aan dat we gemiddeld rond de 10 fietsen fout zitten. Dit is natuurlijk geen geweldig resultaat dus hopelijk kan een machine learning model dit beter dan een simpel wiskundig model.

## Lineaire regressie

Voor we de eigenlijke training doen, moeten we de dataset in een vorm brengen die geaccepteerd wordt door de gebruikte functies. In dit geval maken we gebruik van de [scikit-learn library](https://scikit-learn.org/stable/modules/classes.html), In de documentatie vinden we dat modellen op de volgende manier getraind worden:<br><br>
<em> model = LinearRegression().fit(X, y)</em><br><br>
Hier staat X voor onze features en y voor onze targetwaarde, dus we moeten eerst onze train en test set splitsen.

In [ ]:
train_target = training_df['netBikeChange']
train_df_features = training_df.drop('netBikeChange',axis=1)

test_target = test_df['netBikeChange']
test_df_features = test_df.drop('netBikeChange',axis=1)

Ons eerste model is een lineaire regressie, deze zal proberen de target waarde te voorspellen als een lineaire combinatie van de verschillende features.

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_df_features, train_target)

predictions= lr.predict(test_df_features)
lr_test_MSE = mean_squared_error(test_target,predictions)
print(f"linear regressor: de MSE error op de test set is: {lr_test_MSE}")
lr_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"ridge regressor: de MSE error op de test set is: {lr_test_RMSE}")


We hebben zojuist ons eerste machine learning model kunnen trainen in slechts 2 lijnen code! <br>
Deze is al een verbetering ten opzichte van onze baseline, de gemiddelde fout hebben we kunnen verlagen van 10 fietsen naar 2.5 fietsen.<br>

### Feature importance
Nu we een getraind model hebben kunnen we ook eens kijken welke van onze features het belangrijkste zijn tijdens onze voorspellingen.
Aangezien lineaire regressie gewoon een lineaire combinatie is van onze features kunnen we hun belang vinden gewoon door te kijken naar hun gewichten, hoe groter deze waarde hoe belangrijker de feature.

In [ ]:
coefficients = lr.coef_
sorted_features = sorted(zip(train_df_features.columns, coefficients), key=lambda x: x[1], reverse=True)
print('Feature Coefficients:')
for feature, coefficient in sorted_features:
    print(f'{feature}: {coefficient}')


In [ ]:
def featureImportanceGraph(importanceZip):
    sorted_features = sorted(importanceZip, key=lambda x: x[1], reverse=True)
    features, importance_values=  zip(*sorted_features)
    plt.bar(features, importance_values, color='blue')
    plt.xlabel('Features')
    plt.xticks(features, rotation= 90)
    plt.ylabel('Importance')
    plt.title('Feature Importance')
    fig = plt.gcf()
    plt.show()
    return fig

In [ ]:
_ =featureImportanceGraph(sorted_features)

Als we de coeficcienten extraheren krijgen we een globaal overzicht op het belang van de verschillende features. Via SHAP kunnen we inzoomen en de contributie van de features binnen één voorbeeld bekijken. 

In [ ]:
explainer = shap.LinearExplainer(lr, test_df_features)
shap_values = explainer(test_df_features)
shap.plots.waterfall(shap_values[0])


## Laten we nog wat andere modellen proberen

### Ridge regression

Het volgende model dat we proberen is een variatie op lineaire regressie, namelijk Ridge regressie. Ze werken op een gelijkaardige manier, het verschil is dat Ridge regressie een regularisatie term toevoegt. Deze probeert de verschillende gewichten zo klein mogelijk te houden, wat van pas komt gezien de vorige resultaten. Voor deze regularisatie term is een hyperparameter alpha nodig, deze weegt af tussen de gewichten zo klein mogelijk houden en een accuraat model.<br>

De CV (in RidgeCV) staat voor cross-validatie, dit model kiest volgens deze techniek de beste alpha waarde uit een reeks die wij meegeven zonder dat wij een validatie dataset moeten meegeven.


In [ ]:
from sklearn.linear_model import Ridge
# Experimenteer met de alpha waarde
ridge = Ridge(alpha=0.1)
ridge.fit(train_df_features, train_target)
predictions= ridge.predict(test_df_features)
ridge_test_MSE = mean_squared_error(test_target,predictions)
print(f"ridge regressor: de MSE error op de test set is: {ridge_test_MSE}")
ridge_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"ridge regressor: de MSE error op de test set is: {ridge_test_RMSE}")

coefficients = ridge.coef_
ridge_features = sorted(zip(train_df_features.columns, coefficients), key=lambda x: x[1], reverse=True)
print('Feature Coefficients:')
for feature, coefficient in ridge_features:
    print(f'{feature}: {coefficient}')
_ = featureImportanceGraph(ridge_features)

In [ ]:
from sklearn.linear_model import RidgeCV

alphas = np.logspace(-6, 6, 25)
ridgeCV = RidgeCV(alphas=alphas)
ridgeCV.fit(train_df_features, train_target)
predictions= ridgeCV.predict(test_df_features)
ridge_test_MSE = mean_squared_error(test_target,predictions)
print(f"ridgeCV regressor: de MSE error op de test set is: {ridge_test_MSE}")
ridge_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"ridgeCV regressor: de MSE error op de test set is: {ridge_test_RMSE}")

coefficients = ridgeCV.coef_
ridgeCV_features = sorted(zip(train_df_features.columns, coefficients), key=lambda x: x[1], reverse=True)
print('Feature Coefficients:')
for feature, coefficient in ridgeCV_features:
    print(f'{feature}: {coefficient}')
_ = featureImportanceGraph(ridgeCV_features)

Ondanks dat we geen verbetering merken op de test mean squared error in vergelijking met de gewone lineaire regressie, zijn de gewichten van de verschillende features nu overzichtelijker (zonder extreem grote waardes). Hierdoor krijgen we een beter overzicht welke nu van belang zijn.

In [ ]:
explainer = shap.LinearExplainer(ridgeCV, test_df_features)
shap_values = explainer(test_df_features)
shap.plots.waterfall(shap_values[1])


### Decision tree

Vervolgens kunnen we ook even kijken naar beslissingsbomen (decision trees), deze werken aan de hand van slimme vragen over de features die de data telkens opdeelt in 2 delen. De voorspelling is afhankelijk van het eindpunt waar we op belanden na het beantwoorden van alle vragen.<br><br>
Een belangrijke parameter hier is de diepte van de boom, dit vertelt hoeveel vragen we moeten beantwoorden voor we een antwoord krijgen.<br> 
Experimenteer maar even met deze diepte en kijk naar het effect op MSE (normale dieptes zitten tussen de 1 en 10). Het is ook mogelijk om de boom uiteindelijk te plotten via de plot_tree functie

In [ ]:
from sklearn import tree
#experimenteer met de diepte
decTree = tree.DecisionTreeRegressor(max_depth=10) 
decTree.fit(train_df_features, train_target)
predictions= decTree.predict(test_df_features)
decTree_test_MSE = mean_squared_error(test_target,predictions)
print(f"Decision tree: de MSE error op de test set is: {decTree_test_MSE}")
decTree_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"Decision tree: de MSE error op de test set is: {decTree_test_RMSE}")

feature_importances = decTree.feature_importances_
tree_features = sorted(zip(train_df_features.columns, feature_importances), key=lambda x: x[1], reverse=True)
print('Feature Coefficients:')
for feature, coefficient in tree_features:
    print(f'{feature}: {coefficient}')
_ = featureImportanceGraph(tree_features)

In [ ]:
tree.plot_tree(decTree)

Bij een diepte van 10 zien we al een verbetering in de MSE van 2.5 naar minder dan 2 fietsen. We hebben dus een nieuwe beste versie gevonden! Het plotten van de boom is echter wel onoverzichtelijk geworden.<br>
De belangrijkste features waren uiteindelijk:  de 'verandering van fietsen' tijdens het vorige uur, het uur van de dag, het station en het aantal plaatsen voor fietsen.

In [ ]:
explainer = shap.TreeExplainer(decTree)
shap_values = explainer(test_df_features)
shap.plots.waterfall(shap_values[0])

### Random Forest

Het volgende model dat we gaan uittesten is een random forest regressor. Dit is een ensemble van decision trees (meerdere decision trees), waarbij de resultaten van elke tree samen worden gebracht om het uiteindelijke resultaat te verkrijgen.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Experimenteer met de max_depth (diepte) en n_estimators (aantal bomen) parameters
rf = RandomForestRegressor(max_depth=8, n_estimators=25) 
rf.fit(train_df_features, train_target)
predictions= rf.predict(test_df_features)
rf_test_MSE = mean_squared_error(test_target,predictions)
print(f"Random forest: de MSE error op de test set is: {rf_test_MSE}")
rf_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"Random forest: de MSE error op de test set is: {rf_test_RMSE}")

feature_importances = rf.feature_importances_
rf_features = sorted(zip(train_df_features.columns, feature_importances), key=lambda x: x[1], reverse=True)
print('Feature Coefficients:')
for feature, coefficient in rf_features:
    print(f'{feature}: {coefficient}')
_ = featureImportanceGraph(rf_features)

Ondanks dat we werken met 24 extra decision trees blijft de performantie op de testset hetzelfde als die met slechts 1 boom. Dit is dus geen goeie keuze voor ons uiteindelijk model.<br>
We zien ook terug dezelfde meest belangrijke features 

In [ ]:
explainer = shap.TreeExplainer(rf)
shap_values = explainer(test_df_features)
shap.plots.waterfall(shap_values[0])

### Gradient boosting

Vervolgens kunnen we kijken naar een gradient boost variant. Opnieuw een ensemble van beslissingsbomen zoals random forest, maar hier stemmen de verschillende bomen niet op het eindresultaat. Bij gradient boosting probeert elke boom de fouten van de vorige boom te corrigeren. We krijgen dus een serie van beslissingsbomen
Normale gradient boost modellen bevatten ingebouwde functies om het belang van de features te bekijken, maar dit is niet het geval bij de histogram versie. We zullen deze dus opnieuw berekenen aan de hand van permutaties. Een alternatieve manier om de belangen te berekenen. We veranderen telkens de waardes van 1 feature en controleren de invloed op de performantie, hoe groter de invloed hoe belangrijker de feature.<br>


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.inspection import permutation_importance

ensambleTree = HistGradientBoostingRegressor()
ensambleTree.fit(train_df_features, train_target)
predictions= ensambleTree.predict(test_df_features)
tree_test_MSE = mean_squared_error(test_target,predictions)
print(f"GradientBoosting ensamble: de MSE error op de test set is: {tree_test_MSE}")
tree_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"GradientBoosting ensamble: de MSE error op de test set is: {tree_test_RMSE}")

# Een alternatieve manier om het belang van de features te berekenen
result = permutation_importance(ensambleTree, test_df_features, test_target, n_repeats=5, random_state=42)
feature_importances = result.importances_mean
ensambleTreeFeatures = sorted(zip(train_df_features.columns, feature_importances), key=lambda x: x[1], reverse=True)
for feature, coefficient in ensambleTreeFeatures:
    print(f'{feature}: {coefficient}')
_ = featureImportanceGraph(ensambleTreeFeatures)

Dit model is een verbetering ten opzichte van de vorige, we krijgen nu een gemiddelde fout van 1.6 fietsen!<br>
Er wordt hier ook gebruikgemaakt van verschillende features om het eindresultaat te bekomen, de belangrijkste zijn het uur van de dag, de verandering van fietsen vorig uur en het station zelf.

In [ ]:
explainer = shap.TreeExplainer(ensambleTree)
shap_values = explainer(test_df_features)
shap.plots.waterfall(shap_values[0])


### Multilayer perceptron

We hebben nu al 2 lineaire modellen en 3 beslissingsbomen getraind. Laten we dan eens kijken naar wat complexere modellen, zoals een neuraal netwerk.Meer specifiek een multilayer perceptron (MLP), MLP betekent gewoon verschillende lagen aan neuronen (of perceptrons) en is de simpelste variant van een neuraal netwerk.<br>
De output van iedere neuron is de lineaire combinatie van de outputs van de vorige laag, we voegen non-lineariteit toe door middel van een activatiefunctie toe te passen op de neuron outputs.<br><br>
Dit complexere model geeft meer flexibiliteit maar maakt het ook moeilijker om het belang van de verschillende features te vinden (black box). 

In [ ]:
from sklearn.neural_network import MLPRegressor
# Test verschillende netwerk groottes en dieptes (Gelieve deze wel redelijk te houden zodat de cluster dit aankan)
mlp = MLPRegressor(hidden_layer_sizes=(32, 16,8),random_state=1, max_iter=5)
mlp.fit(train_df_features, train_target)
predictions= mlp.predict(test_df_features)
mlp_test_MSE = mean_squared_error(test_target,predictions)
print(f"Multilayer perceptron: de MSE error op de test set is: {mlp_test_MSE}")
mlp_test_RMSE = root_mean_squared_error(test_target,predictions)
print(f"Multilayer perceptron: de MSE error op de test set is: {mlp_test_RMSE}")

# Een alternatieve manier om het belang van de features te berekenen
result = permutation_importance(mlp, test_df_features, test_target, n_repeats=5, random_state=42)
feature_importances = result.importances_mean
MlpFeatures = sorted(zip(train_df_features.columns, feature_importances), key=lambda x: x[1], reverse=True)
for feature, coefficient in MlpFeatures:
    print(f'{feature}: {coefficient}')
_ = featureImportanceGraph(MlpFeatures)

Ondanks dat we een complexer, non-lineair model gebruiken zien we geen verbetering in performance, integendeel zelfs. Een zo complex mogelijk model op een probleem gooien is dus niet altijd de oplossing.<br>
De verandering van fietsen tijdens het vorige uur is hier veruit het belangrijkste feature.

# Visualiseer de resultaten

Nu hebben we enkele modellen getraind en klaar voor gebruik binnen het beachboys bikesharing bedrijf. Alhoewel sommige van deze modellen al een redelijke prestatie halen, is er altijd ruimte voor verbetering. Net zoals bij de start van de workshop, beginnen we hier aan door inzichten te krijgen in de data. Hier zijn nu de voorspellingen gemaakt door de verschillende modellen bij gekomen. We kunnen eens kijken hoe deze afwegen tegen de realiteit, en zo leren waar ze in de mist gaan.

Eerst kiezen we een bepaalde dag en station uit de test set om te plotten (tussen 1 juni en 14 augustus):

In [ ]:
# Experimenteer maar met deze waardes
station = 70 #dit is een vrij druk station
maand = 8
dag= 11

viz_data = test_df[(test_df['Station']== station)&(test_df['day']== dag)&(test_df['month']== maand)]
viz_data_target = viz_data['netBikeChange']
viz_data_features = viz_data.drop('netBikeChange',axis=1)



De volgende code genereert een plot voor ons, we kiezen zelf gewooon het te gebruiken model.

**Model type: variabele met getraind model**<br>
Lineaire regressie: lr <br>
Ridge regression: ridgeCV <br>
Decision tree: decTree<br>
Random forest: rf <br>
Gradient Boosting: ensambleTree <br>
multilayer perceptron: mlp <br>

In [ ]:
# test verschillende modellen uit
model = lr

predictions = model.predict(viz_data_features)
plt.figure(figsize=(8, 6))
plt.plot(viz_data_features['hour'],predictions, label ='Predictions',color='r')
plt.bar(viz_data_features['hour'], viz_data_target, label='Actual')
plt.xlabel('Uur van de dag')
plt.xticks(viz_data_features['hour'])
plt.ylabel('Verandering van fietsen')
plt.title(f'Voorspelling vs Realiteit')
plt.grid(True, linestyle='--', alpha=0.7)

# Adding origin lines (axes lines)
plt.axhline(0, color='black',linewidth=0.5)
plt.legend()
plt.show()


# Modellen evalueren
MLflow is een open-source platform dat helpt bij het ontwikkelen, opzetten en onderhouden van machine learning modellen. Het creeert een overview van een specifiek model en kan verschillende zaken bijhouden. Zoals het model zelf, verschillende parameters gebruikt om het model af te stellen, en meetwaardes gevonden tijdens het trainen en testen van het model. <br>
Op deze manier kunnen we een duidelijk overzicht houden van alle modellen, alsook vergelijking maken tussen de verschillende versies.<br>
MLflow heeft een eigen UI die lokaal geopend kan worden, en is ook geintegreerd met Azure ML. <br>
MLflow werkt met runs, een run is één werkprocess dat we gaan opvangen, dit kan het trainen van een bepaald model zijn of gewoon enkele parameters. Deze runs kunnen we dan groeperen binnen een experiment om te vergelijken.<br>
Binnen AzureML, vinden we in de jobs tab, al onze MLflow experimenten en runs.

In [ ]:
import mlflow
from mlflow.models import infer_signature


## Een eerste run loggen

In [ ]:
experiment_name = "workshop_v1"
mlflow.set_experiment(experiment_name)

# Start een run, het trainen van 1 model
with mlflow.start_run(run_name= "First Test"):
    Name = "Bram"
    mlflow.log_param("Test Param",Name)

## Een model manueel loggen
We gaan de eerdere decicion tree nogmaals trainen en deze keer gaan we manueel de verschillende parameters en meetwaarden opslaan

In [ ]:
# Start een experiment, groupeert verschillende runs
experiment_name = "workshop_v1"
mlflow.set_experiment(experiment_name)

# Start een run, het trainen van 1 model
with mlflow.start_run(run_name= "Tree_Manual"):

    # De hyperparameter die we gaan opslaan
    depth = 10
    mlflow.log_param("Tree depth", depth)

    # Gekopieerde code van de vorige modellen
    decTree = tree.DecisionTreeRegressor(max_depth=depth) 
    decTree.fit(train_df_features, train_target)
    predictions= decTree.predict(test_df_features)
    
    # dit is hoe we manueel ons model opslaan, een makkelijkere manier zien we direct
    #signature = infer_signature(train_df_features, decTree.predict(train_df_features))
    #mlflow.sklearn.log_model(decTree, "decision_tree_regressor", signature=signature)

    decTree_test_MSE = mean_squared_error(test_target,predictions)
    print(f"Decision tree: de MSE error op de test set is: {decTree_test_MSE}")
    mlflow.log_param("test_MSE",decTree_test_MSE) # Neem ook deze parameter mee in het overzicht
    
    decTree_test_RMSE = root_mean_squared_error(test_target,predictions)
    print(f"Decision tree: de RMSE error op de test set is: {decTree_test_RMSE}")
    mlflow.log_param("test_MSE",decTree_test_MSE) # Neem ook deze parameter mee in het overzicht


## een model automatisch loggen
MLflow laat ook automatisch loggen toe

In [ ]:
# Start een experiment, groupeert verschillende runs
experiment_name = "workshop_v1"
mlflow.set_experiment(experiment_name)

# Start een run, het trainen van 1 model
with mlflow.start_run(run_name= "Tree_autolog_Only"):
    #Log automatisch alle zaken over het model
    mlflow.sklearn.autolog()

    depth = 10
    # Gekopieerde code van de vorige modellen
    decTree = tree.DecisionTreeRegressor(max_depth=depth) 
    decTree.fit(train_df_features, train_target)
    predictions= decTree.predict(test_df_features)
    
    decTree_test_MSE = mean_squared_error(test_target,predictions)
    print(f"Decision tree: de MSE error op de test set is: {decTree_test_MSE}")

    decTree_test_RMSE = root_mean_squared_error(test_target,predictions)
    print(f"Decision tree: de RMSE error op de test set is: {decTree_test_RMSE}")


## Combineer beide

In [ ]:
# Start een experiment, groupeert verschillende runs
experiment_name = "workshop_v1"
mlflow.set_experiment(experiment_name)

# Start een run, het trainen van 1 model
with mlflow.start_run(run_name= "Tree_autolog_v1"):
    mlflow.sklearn.autolog() #Log automatisch alle zaken over het model

    depth = 10
    mlflow.log_param("Tree depth", depth)
    
    # Gekopieerde code van de vorige modellen
    decTree = tree.DecisionTreeRegressor(max_depth=depth) 
    decTree.fit(train_df_features, train_target)
    predictions= decTree.predict(test_df_features)
    
    decTree_test_MSE = mean_squared_error(test_target,predictions)
    print(f"Decision tree: de MSE error op de test set is: {decTree_test_MSE}")
    mlflow.log_param("test_MSE",decTree_test_MSE) # Neem ook deze parameter mee in het overzicht

    decTree_test_RMSE = root_mean_squared_error(test_target,predictions)
    print(f"Decision tree: de RMSE error op de test set is: {decTree_test_RMSE}")
    mlflow.log_param("test_RMSE",decTree_test_RMSE) # Neem ook deze parameter mee in het overzicht

    feature_importances = decTree.feature_importances_
    tree_features = sorted(zip(train_df_features.columns, feature_importances), key=lambda x: x[1], reverse=True)
    fig = featureImportanceGraph(tree_features)
    mlflow.log_figure(fig, "feature_importance.png") # Neem ook de grafiek mee in het overzicht

## Multi step training proces


In [ ]:
experiment_name = "workshop_v1_training_logging"
mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name= "MLP_splitTraining_v1"):
  mlflow.sklearn.autolog()
  mlp = MLPRegressor(hidden_layer_sizes=(8, 4),random_state=1, max_iter=1)
  num_epochs = 20
  for epoch in range(num_epochs):
     mlp.partial_fit(train_df_features, train_target)
     y_pred = mlp.predict(train_df_features)
     mse = mean_squared_error(train_target, y_pred)
     mlflow.log_metric("train_mean_squared_error", mse)
     
  predictions= mlp.predict(test_df_features)
  mlp_test_MSE = mean_squared_error(test_target,predictions)
  print(f"Gradient descent: de MSE error op de test set is: {mlp_test_MSE}")
  mlflow.log_metric("test_mean_squared_error", mlp_test_MSE)

## Hyperparameter tuning

In [ ]:
hidden_layers = [(8,4),(16,8),(4,2)]
for hidden_layer in hidden_layers:
    with mlflow.start_run(run_name= f"MLP_hyper_{hidden_layer}"):
        mlflow.sklearn.autolog()
        mlp =  MLPRegressor(hidden_layer_sizes=hidden_layer,random_state=1, max_iter=15)
        mlp.fit(train_df_features, train_target)
        predictions= mlp.predict(test_df_features)
        mlp_test_MSE = mean_squared_error(test_target,predictions)
        print(f"Gradient descent: de MSE error op de test set is: {mlp_test_MSE}")
        mlflow.log_metric("test_mean_squared_error", mlp_test_MSE)

## Nu is het aan jullie

In [ ]:
# Start een experiment, groupeert verschillende runs
experiment_name = "workshop_v1_sandbox"
mlflow.set_experiment(experiment_name)

# Start een run, het trainen van 1 model
with mlflow.start_run(run_name= "sandbox_v1"):
    #Log automatisch alle zaken over het model
    mlflow.sklearn.autolog() 
    
    #log parameters
    mlflow.log_param("", )
    
    # Train een model
  
    
    #log meetwaardes
    mlflow.log_param("",) 

